<a href="https://www.kaggle.com/code/ocanaydin/meals-recognition-tfl-basic?scriptVersionId=113933745" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Access your google drive.

## IMPORTANT!!!

I highly recommend that you should use this notebook in google colab.This is a basic implemantation of portuguese meals recognition problem.I tried to explain every part of model.If you have any comments or suggestions about this notebook please write it.Enjoy!!!

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# **Step 1** : Send images to google drive as their categories.

### (1)Create train and validation directories in drive.

In [ ]:
import os
"""This is your base directory path.Indeed,it indicates where you want to put all the images as their directories."""
"""Below is an example path which i use for this project."""
base_dir = "/content/drive/My Drive/Portuguese_Meals_Recognition_Test"
directories = ["train","validation"]
"""We have two directories in this project as shown above.We can create new directories as train and validation in base dir."""
for i in range(len(directories)):
  folder = os.path.join(base_dir,directories[i])
  os.mkdir(folder)

###(2)Extract zip file to your base directory.

In [ ]:
import zipfile as zf
"""First download zip file your pc.After that you can upload zip file to your base directory using drive."""
"""I have changed the name of zip file.You can use name whatever you want."""
files = zf.ZipFile(os.path.join(base_dir,"meals.zip"),"r")
files.extractall(base_dir)
files.close()
"""Now you can see all images in your drive with their directories."""

###(3)Create subdirectories with class names in train and validation directories.

In [ ]:
"""We define images path which we extracted in second step.You can see portuguese_food_2 folder in your drive"""
main_images_path = os.path.join(base_dir,"portuguese_food_2")
"""We can get the class names from their directory names."""
classes = os.listdir(main_images_path)

In [ ]:
"""Here,we create a subdirectory in both train and validation directories for every class name."""
for i in range(len(directories)):
  for j in range(len(classes)):
    folder = os.path.join(base_dir,directories[i])
    folder = os.path.join(folder,classes[j])
    os.mkdir(folder)

###(4)Copy all images to train and validation directories as their belonged class. (Train has %80 ,Validation %20)

In [ ]:
import shutil
"""Here we see 3 path.First is assigned above but we can assign it again.Train path and validation path are our directories which we want to copy images to them."""
main_images_path = os.path.join(base_dir,"portuguese_food_2")
train_path = os.path.join(base_dir,"train")
validation_path = os.path.join(base_dir,"validation")
"""Here i have advices for you.First use train_path in below loop and define range in 0 to int(n * 0.8) as given. (Train images have to be %80 of all images.)"""
"""Secondly,you can change train_path to validation_path.And you should also change to range in int(n * 0.8 ) to int(n) (Validation images have to be %20 of all images.)"""
for filename in os.listdir(main_images_path):
  images_path = os.path.join(main_images_path,filename)
  n = len(os.listdir(images_path))
  for i in range(int(n * 0.8),int(n)):
    src = os.path.join(images_path,os.listdir(images_path)[i])
    dst = os.path.join(validation_path,filename)
    shutil.copy(src,dst)

###(5)Ensure that images whether any of them are corrupted.

In [ ]:
from os import listdir
from PIL import Image
def check_corrupted(directory_type,class_name):
  path = base_dir+"/{}/{}/".format(directory_type,class_name)
  for filename in listdir(path):
      if filename.endswith('.jpg'):
          try:
              img = Image.open(path+filename)  # open the image file
              img.verify()  # verify that it is, in fact an image
          except (IOError, SyntaxError) as e:
              print(filename)
              os.remove(path+filename)

In [ ]:
"""Use checking function in below for both train and validation directories.If you see corrupted image,you can delete it."""
for i in range(len(directories)):
  for j in range(len(classes)):
    check_corrupted(directories[i],classes[j])

### (6)See the numbers of images in directories.

In [ ]:
for i in range(len(classes)):
  print(classes[i])
  print(len(os.listdir(os.path.join(train_path,classes[i]))))
  print(len(os.listdir(os.path.join(validation_path,classes[i]))))

#**Step 2** : Image Data Generator for training and validation.

In [ ]:
import tensorflow as tf

###(1)Training

In [ ]:
import os
"""You dont need to reassing these two pth but i done it for convenience."""
base_dir = "/content/drive/My Drive/Portuguese_Meals_Recognition_Test"
train_path = os.path.join(base_dir,"train")
"""All images will be scaled by 1/255 to normalize.(pixels between 0-1).Also image augmentation is used."""
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)
"""We can create our train generator."""
train_generator = train_datagen.flow_from_directory(train_path,target_size = (300,300),class_mode = "categorical",batch_size = 128)

In [ ]:
validation_path = os.path.join(base_dir,"validation")
"""All images will be scaled by 1/255 to normalize.(pixels between 0-1).Also image augmentation is NOT used for validation dataset."""
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255
)
"""We can create our validation generator."""
validation_generator = validation_datagen.flow_from_directory(validation_path,target_size = (300,300),class_mode = "categorical",batch_size = 128)

#**Step 3** : CNN Archictecture with Transfer Learning

###(1)Use InceptionV3 as base model.

In [ ]:
"""Here i used InceptionV3 for base model.You can use other models if you want."""
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (300,300,3),include_top = False,weights = "imagenet")
"""Freeze all layers to stop updating InceptionV3 trained weights. """
for layer in base_model.layers:
  layer.trainable = False

###(2)Create an architecture to feed model with images.

In [ ]:
import tensorflow 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers 


In [ ]:
"""Here we can get the last layer as mixed8.It means that we can start updating our weights from mixed8 layer.You can change it if you want."""
last_layer = base_model.get_layer("mixed8")
print(last_layer.output_shape)
"""Flatten layer to reduce output shape to 1 dim."""
x = layers.Flatten()(last_layer.output)
"""Add fully connected layers to 512 units."""
x = layers.Dense(512,activation = "relu")(x)
"""Add dropout layer."""
x = layers.Dropout(0.2)(x)
"""Output Layer which contains number of classes units (23)"""
classes = os.listdir(train_path)
x = layers.Dense(len(classes),activation = "softmax")(x)
"""Here we can connect our model end to end."""
model = tensorflow.keras.models.Model(base_model.input,x)

In [ ]:
model.summary()

#**Step 4** :Compile and fit the model

In [ ]:
"""I used RMSprop optimizer here.You can change if you want."""
model.compile(RMSprop(learning_rate = 0.001),loss = "categorical_crossentropy",metrics = ["acc"])

In [ ]:
"""This is our callback class.Using this callback,we guaranteed if accuracy of model > 0.95,training will be stopped.It prevents overfitting."""
class MyCallback(tensorflow.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs = {}):
    if logs.get("acc") >= 0.95:
      print("Model reached accuracy %95.Stop it.")
      self.model.stop_training = True
callback = MyCallback()
"""Here we can start train model."""
history = model.fit(train_generator,epochs = 20,batch_size = 128,validation_data = validation_generator,verbose = 1,callbacks = [callback])

# **Step 5** : Plot accuracies and losses of model

In [ ]:
"""After training we reached %91.62 accuracy without overfitting.It is not a bad result.Here we can draw to compare training and validation metrics."""
import matplotlib.pyplot as plt 
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(len(acc)) 

In [ ]:
plt.plot(epochs,acc,"r",label = "Training Accuracy")
plt.plot(epochs,val_acc,"b",label = "Validation Accuracy")
plt.title("Accuracies")
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs,loss,"r",label = "Training Loss")
plt.plot(epochs,val_loss,"b",label = "Validation Loss")
plt.title("Losses")
plt.legend()
plt.show()

#**Step 6** : Save model

In [ ]:
"""You should save your model to do not train model again and again.You should assign a part.I change base dir which is assigned above."""
model.save(os.path.join(base_dir,"portuguese_meals_recognition_TFL.h5"))

# Load Model

In [ ]:
"""You can import your saved model to reuse."""
model1 = tf.keras.models.load_model(os.path.join(base_dir,"portuguese_meals_recognition_TFL.h5"))

#**Step 7** : Get the images from internet and process them.

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from tensorflow.keras.preprocessing import image  
import numpy as np
def get_and_process(url):
  """We should send a request to see our chosen image from google."""
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  """img1 is defined to show image using its original shape."""
  img1 = img
  """Resize image for appropriate input shape for model. """
  img = img.resize((300,300))
  """Convert img to numpy array,rescale it,expand dims for model input shape convenince and check vertically."""
  x = image.img_to_array(img)
  x = x / 255.0
  x = np.expand_dims(x,axis = 0)
  img_tensor = np.vstack([x])
  """Return original image and img tensor which we will be using to find prediction of image."""
  return img1,img_tensor 

#**Final** : Predict Image

In [ ]:
import matplotlib.pyplot as plt 
classes = os.listdir(os.path.join(base_dir,"train"))
"""Here you can select an image and copy its url to here.It must be involved in 23 classes."""
url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTtaiSaYU4-6O2BvK7YDdo57BKdzW5ySUTxlQ&usqp=CAU"
"""Use our above function to process img and predict it."""
img1,test_img = get_and_process(url)
pred = model1.predict(test_img)

"""Here you can see that pred includes 23 probability numbers.They are here because we used softmax as an optimizer in last layer.We can get index of max probability in pred list."""
"""This index gives us our predicted class."""
print(f"Prediction : {classes[np.argmax(pred)]}")
"""Show image."""
plt.imshow(img1)
plt.show()
print(classes)